In [17]:
import pandas as pd
import numpy as np
import boto3
import psycopg2
import configparser

In [18]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

In [19]:
RDS_HOST = 'tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com'
print(RDS_HOST)

tienda-transactional.cjykcg6gmirx.us-east-2.rds.amazonaws.com


In [20]:
postgres_driver = f"""postgresql://{config.get('TRANSACC', 'DB_USER')}:{config.get('TRANSACC', 'DB_PASSWORD')}@{RDS_HOST}:{config.get('TRANSACC', 'DB_PORT')}/{config.get('TRANSACC', 'DB_NAME')}"""

#### Creamos Dimensión de Personas


In [21]:
sql_query = 'SELECT * FROM persona;'
dimPersonas = pd.read_sql(sql_query, postgres_driver)
dimPersonas.head()

,idpersona,tipo_persona,nombre,tipo_documento,num_documento,direccion,telefono,email
0,565399554,cliente,Michelle Perez,Tarjeta Residencia,176111188549635,Unit 1817 Box 2185 DPO AA 64429,516.780.,vrodriguez@gmail.com
1,665966595,cliente,Debbie Benjamin,DNI,168670898616328,"42008 Brandon Union Cookhaven, LA 92481",616-765-,wvalentine@hotmail.com
2,783800325,cliente,Allen Odonnell,Pasaporte,110302549389322,"076 Jasmine Islands Smithtown, DE 24081",(452)499,kimberly50@gmail.com
3,826793991,cliente,Matthew Li,Tarjeta Residencia,156352804720651,"736 Lee Causeway South Brandonland, OK 43735",87370337,qhampton@gmail.com
4,504653842,cliente,Chad Lopez,DNI,148246596558860,"04003 French Manor Suite 701 East Yolanda, AL ...",821.348.,hannah97@yahoo.com


#### Dimensión de Usuario


In [22]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario = df_usuario.rename(columns={'nombre': 'nombre_usuario'})
df_usuario.head()

,idusuario,idrol,nombre_usuario,tipo_documento,num_documento,direccion,telefono,email,clave,estado
0,8818688,10003,Richard James,DNI,158643840073728,"222 Obrien Ferry West Williamland, MP 22317",515-922-,ikramer@yahoo.com,"[b'c', b'0', b'2', b'2', b'f', b'3', b'5', b'1...",1
1,7258115,10003,Amber Grant,Pasaporte,125079704406019,"76095 Wagner Ford Apt. 141 Ryanmouth, VI 68589",552-971-,smitheugene@gmail.com,"[b'd', b'8', b'7', b'5', b'd', b'c', b'7', b'8...",1
2,5206026,10002,Carolyn Jones,NIE,115369222688772,"85590 Pamela Lights Apt. 211 Joyceville, NJ 92480",+1-914-4,aalvarez@gmail.com,"[b'7', b'4', b'4', b'6', b'a', b'9', b'c', b'9...",0
3,2824202,10002,Brandy Bryant,NIE,133877029232646,"5870 Massey Grove Suite 074 Lake Jessicaville,...",+1-772-5,odonnellashley@yahoo.com,"[b'f', b'5', b'5', b'5', b'0', b'1', b'd', b'8...",0
4,4433933,10004,Tom Salinas,Pasaporte,137802450690055,"49057 Garcia Junctions Smithberg, KS 74217",(865)418,lisawhitaker@hotmail.com,"[b'1', b'c', b'3', b'9', b'9', b'3', b'4', b'6...",0


In [23]:
sql_query = 'SELECT * FROM rol;'
df_rol = pd.read_sql(sql_query, postgres_driver)
df_rol = df_rol.rename(columns={'nombre': 'nombre_rol'})
df_rol.head()

,idrol,nombre_rol,descripcion,estado
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


In [24]:
df_usuario = df_usuario.merge(df_rol, how='inner', on='idrol')
dimUsuario = df_usuario.drop(['idrol'], axis=1)
dimUsuario.head()

,idusuario,nombre_usuario,tipo_documento,num_documento,direccion,telefono,email,clave,estado_x,nombre_rol,descripcion,estado_y
0,8818688,Richard James,DNI,158643840073728,"222 Obrien Ferry West Williamland, MP 22317",515-922-,ikramer@yahoo.com,"[b'c', b'0', b'2', b'2', b'f', b'3', b'5', b'1...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
1,7258115,Amber Grant,Pasaporte,125079704406019,"76095 Wagner Ford Apt. 141 Ryanmouth, VI 68589",552-971-,smitheugene@gmail.com,"[b'd', b'8', b'7', b'5', b'd', b'c', b'7', b'8...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
2,9232398,Tracey Turner,NIE,187735268761606,15792 Jeffrey Harbors Suite 399 Lake Davidfort...,887.944.,nle@hotmail.com,"[b'8', b'6', b'2', b'd', b'7', b'3', b'9', b'd...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,6924309,Heather Nolan,NIE,103651294844940,"2546 William Point Apt. 872 New Ryan, OK 24437",(581)451,mccalljoshua@gmail.com,"[b'6', b'5', b'9', b'2', b'0', b'5', b'7', b'e...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
4,7155738,James Schneider,Pasaporte,197192471740432,4727 Jennifer Springs Apt. 717 West Barbaravie...,001-825-,alex68@hotmail.com,"[b'6', b'e', b'a', b'4', b'5', b'b', b'f', b'b...",1,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1


#### Dimensión Artículo


In [25]:
sql_query = 'SELECT * FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'nombre': 'nombre_articulo', 'descripcion': 'descripcion_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre_articulo,precio_venta,stock,descripcion_articulo,imagen,estado
0,299008,97521,4GlHzDRZ,Camisa Since,554.69,18,Artículo Camisa Since en venta en tienda,fig_0,1
1,417793,82016,BGkdWS9S,Pelota Hour,748.36,38,Artículo Pelota Hour en venta en tienda,fig_1,1
2,389124,13750,XFsyQfgJ,Pelota Just,514.84,52,Artículo Pelota Just en venta en tienda,fig_2,1
3,233476,51470,0VKtCJNX,Portátil Force,728.38,58,Artículo Portátil Force en venta en tienda,fig_3,1
4,827400,51470,GuqF2Y9Y,Juguete Style,444.59,92,Artículo Juguete Style en venta en tienda,fig_4,1


In [26]:
sql_query = 'SELECT * FROM categoria;'
df_categoria = pd.read_sql(sql_query, postgres_driver)
df_categoria = df_categoria.rename(columns={'nombre': 'nombre_categoria', 'descripcion': 'descripcion_categoria'})
df_categoria.head()

,idcategoria,nombre_categoria,descripcion_categoria,estado
0,39582,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,82016,Calzado,"Incluye zapatos formales, tenis, botas, sandal...",1
2,51470,Juguetes y Juegos,Una amplia gama de juguetes para ninos de toda...,1
3,75369,Electronicos,"Desde telefonos moviles, computadoras portatil...",1
4,20934,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1


In [27]:
df_articulo = df_articulo.merge(df_categoria, how='inner', on='idcategoria')
dimArticulo = df_articulo.drop(['idcategoria'], axis=1)
dimArticulo.head()

,idarticulo,codigo,nombre_articulo,precio_venta,stock,descripcion_articulo,imagen,estado_x,nombre_categoria,descripcion_categoria,estado_y
0,299008,4GlHzDRZ,Camisa Since,554.69,18,Artículo Camisa Since en venta en tienda,fig_0,1,Joyeria,"Desde anillos, pulseras y collares hasta reloj...",1
1,600073,e5WzgI8T,Lámpara Control,816.39,21,Artículo Lámpara Control en venta en tienda,fig_5,1,Joyeria,"Desde anillos, pulseras y collares hasta reloj...",1
2,434193,gCExXOcL,Pantalón Business,966.62,30,Artículo Pantalón Business en venta en tienda,fig_9,1,Joyeria,"Desde anillos, pulseras y collares hasta reloj...",1
3,585745,bMO1JplI,Bolígrafo Memory,305.64,52,Artículo Bolígrafo Memory en venta en tienda,fig_10,1,Joyeria,"Desde anillos, pulseras y collares hasta reloj...",1
4,813083,CuPT6hCy,Jarrón Present,42.68,38,Artículo Jarrón Present en venta en tienda,fig_15,1,Joyeria,"Desde anillos, pulseras y collares hasta reloj...",1


#### Dimensión Venta


In [28]:
sql_query = 'SELECT * FROM venta;'
df_venta = pd.read_sql(sql_query, postgres_driver)
df_venta = df_venta.rename(columns={'estado': 'estado_venta'})
df_venta.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,804864,922565727,4919286,Nota de Crédito,7337987,13783041,2019-06-26,66.80,78626.93,Completada
1,223233,410170198,1642528,Nota de Crédito,6815748,13887489,2022-12-03,18.21,5128.87,Completada
2,727041,837562345,5222074,Recibo,9093126,68136964,2015-09-19,23.11,3700.99,Completada
3,647168,323201389,9489165,Recibo,9406470,47261701,2017-01-13,21.06,13558.78,Reembolsada
4,514056,350764427,8768679,Factura,8867849,61259781,2019-05-24,31.81,14911.36,Reembolsada


In [29]:
sql_query = 'SELECT * FROM persona'
df_cliente = pd.read_sql(sql_query, postgres_driver)
df_cliente = df_cliente[df_cliente['tipo_persona']=='cliente']
df_cliente = df_cliente.rename(columns={'idpersona': 'idcliente', 'nombre': 'nombre_cliente', 'tipo_documento': 'tipo_documento_cliente', 'num_documento': 'num_documento_cliente','direccion': 'direccion_cliente', 'telefono': 'telefono_cliente', 'email': 'email_cliente'})
df_cliente.head()

,idcliente,tipo_persona,nombre_cliente,tipo_documento_cliente,num_documento_cliente,direccion_cliente,telefono_cliente,email_cliente
0,565399554,cliente,Michelle Perez,Tarjeta Residencia,176111188549635,Unit 1817 Box 2185 DPO AA 64429,516.780.,vrodriguez@gmail.com
1,665966595,cliente,Debbie Benjamin,DNI,168670898616328,"42008 Brandon Union Cookhaven, LA 92481",616-765-,wvalentine@hotmail.com
2,783800325,cliente,Allen Odonnell,Pasaporte,110302549389322,"076 Jasmine Islands Smithtown, DE 24081",(452)499,kimberly50@gmail.com
3,826793991,cliente,Matthew Li,Tarjeta Residencia,156352804720651,"736 Lee Causeway South Brandonland, OK 43735",87370337,qhampton@gmail.com
4,504653842,cliente,Chad Lopez,DNI,148246596558860,"04003 French Manor Suite 701 East Yolanda, AL ...",821.348.,hannah97@yahoo.com


In [30]:
df_venta_cliente = df_venta.merge(df_cliente, how='inner', on='idcliente')
df_venta_cliente.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,tipo_persona,nombre_cliente,tipo_documento_cliente,num_documento_cliente,direccion_cliente,telefono_cliente,email_cliente
0,727041,837562345,5222074,Recibo,9093126,68136964,2015-09-19,23.11,3700.99,Completada,cliente,Laura Beard,Pasaporte,177431215044578,"7221 Hunter Motorway Danielville, SC 55626",65751144,umartin@gmail.com
1,690909,837562345,1312428,Factura,1352458,54866679,2016-06-20,69.77,74494.66,Completada,cliente,Laura Beard,Pasaporte,177431215044578,"7221 Hunter Motorway Danielville, SC 55626",65751144,umartin@gmail.com
2,647168,323201389,9489165,Recibo,9406470,47261701,2017-01-13,21.06,13558.78,Reembolsada,cliente,Dr. Matthew Mcclure MD,Pasaporte,159242957130126,USS Foley FPO AP 29501,001-663-,martindennis@yahoo.com
3,864751,323201389,2173097,Nota de Crédito,9445937,19759656,2018-04-05,85.02,55667.16,Cancelada,cliente,Dr. Matthew Mcclure MD,Pasaporte,159242957130126,USS Foley FPO AP 29501,001-663-,martindennis@yahoo.com
4,768827,323201389,6703229,Nota de Crédito,2171768,26893154,2018-09-10,4.99,41374.10,Reembolsada,cliente,Dr. Matthew Mcclure MD,Pasaporte,159242957130126,USS Foley FPO AP 29501,001-663-,martindennis@yahoo.com


In [31]:
sql_query = 'SELECT * FROM usuario;'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_usuario = df_usuario.rename(columns={'nombre': 'nombre_usuario', 'tipo_documento': 'tipo_documento_usuario', 'num_documento': 'num_documento_usuario','direccion': 'direccion_usuario', 'telefono': 'telefono_usuario', 'email': 'email_usuario'})
df_usuario.head()

,idusuario,idrol,nombre_usuario,tipo_documento_usuario,num_documento_usuario,direccion_usuario,telefono_usuario,email_usuario,clave,estado
0,8818688,10003,Richard James,DNI,158643840073728,"222 Obrien Ferry West Williamland, MP 22317",515-922-,ikramer@yahoo.com,"[b'c', b'0', b'2', b'2', b'f', b'3', b'5', b'1...",1
1,7258115,10003,Amber Grant,Pasaporte,125079704406019,"76095 Wagner Ford Apt. 141 Ryanmouth, VI 68589",552-971-,smitheugene@gmail.com,"[b'd', b'8', b'7', b'5', b'd', b'c', b'7', b'8...",1
2,5206026,10002,Carolyn Jones,NIE,115369222688772,"85590 Pamela Lights Apt. 211 Joyceville, NJ 92480",+1-914-4,aalvarez@gmail.com,"[b'7', b'4', b'4', b'6', b'a', b'9', b'c', b'9...",0
3,2824202,10002,Brandy Bryant,NIE,133877029232646,"5870 Massey Grove Suite 074 Lake Jessicaville,...",+1-772-5,odonnellashley@yahoo.com,"[b'f', b'5', b'5', b'5', b'0', b'1', b'd', b'8...",0
4,4433933,10004,Tom Salinas,Pasaporte,137802450690055,"49057 Garcia Junctions Smithberg, KS 74217",(865)418,lisawhitaker@hotmail.com,"[b'1', b'c', b'3', b'9', b'9', b'3', b'4', b'6...",0


In [32]:
df_venta_cliente_usuario = df_venta_cliente.merge(df_usuario, how='inner', on='idusuario')
dimVenta = df_venta_cliente_usuario.drop(['idcliente', 'idusuario', 'idrol'], axis=1)
dimVenta.head()

,idventa,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,tipo_persona,nombre_cliente,...,telefono_cliente,email_cliente,nombre_usuario,tipo_documento_usuario,num_documento_usuario,direccion_usuario,telefono_usuario,email_usuario,clave,estado
0,727041,Recibo,9093126,68136964,2015-09-19,23.11,3700.99,Completada,cliente,Laura Beard,...,65751144,umartin@gmail.com,Adrienne Marsh,DNI,177791090183864,"64905 Benjamin Road West Ryan, KS 59192",972-631-,seth50@hotmail.com,"[b'd', b'4', b'7', b'3', b'9', b'f', b'b', b'6...",1
1,690909,Factura,1352458,54866679,2016-06-20,69.77,74494.66,Completada,cliente,Laura Beard,...,65751144,umartin@gmail.com,Crystal Hess,NIE,127672283920040,"609 Justin Rapids Apt. 062 Port Timothy, PW 32898",225-495-,popevictor@gmail.com,"[b'f', b'c', b'6', b'e', b'3', b'a', b'5', b'f...",1
2,221665,Boleta de Venta,4213267,37640699,2019-06-30,64.01,78140.08,En proceso,cliente,James Ballard,...,662.577.,roy32@yahoo.com,Crystal Hess,NIE,127672283920040,"609 Justin Rapids Apt. 062 Port Timothy, PW 32898",225-495-,popevictor@gmail.com,"[b'f', b'c', b'6', b'e', b'3', b'a', b'5', b'f...",1
3,647168,Recibo,9406470,47261701,2017-01-13,21.06,13558.78,Reembolsada,cliente,Dr. Matthew Mcclure MD,...,001-663-,martindennis@yahoo.com,Timothy Gutierrez,Pasaporte,175150685213433,"933 Michael River Suite 045 Weststad, ID 82260",620.518.,shannon78@yahoo.com,"[b'0', b'2', b'0', b'b', b'5', b'8', b'8', b'8...",1
4,731224,Factura,5976170,27361356,2016-12-07,38.23,26262.48,En proceso,cliente,Karen Sims,...,290.805.,rachel73@yahoo.com,Timothy Gutierrez,Pasaporte,175150685213433,"933 Michael River Suite 045 Weststad, ID 82260",620.518.,shannon78@yahoo.com,"[b'0', b'2', b'0', b'b', b'5', b'8', b'8', b'8...",1


#### Dimensión Ingreso


In [33]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado
0,531619841,376765211,5692000,Ticket de Venta,1916930,None,2023-06-24,71.55,11050.94,Cancelada
1,281835522,742713186,2735407,Recibo,1511426,None,2019-10-17,66.65,74033.63,Reembolsada
2,501905412,478520944,9255184,Ticket de Venta,1388548,None,2020-06-20,94.38,3490.30,Completada
3,781879301,761635125,3604790,Nota de Débito,1478661,None,2017-06-05,33.96,89249.26,Completada
4,900507658,409456127,9170192,Boleta de Venta,2965510,None,2018-12-26,3.13,68756.65,Cancelada


In [34]:
sql_query = 'SELECT * FROM persona;'
df_proveedor = pd.read_sql(sql_query, postgres_driver)
df_proveedor = df_proveedor[df_proveedor['tipo_persona']=='proveedor']
df_proveedor = df_proveedor.rename(columns={'idpersona': 'idproveedor', 'nombre': 'nombre_proveedor', 'tipo_documento': 'tipo_documento_proveedor', 'num_documento': 'num_documento_proveedor', 'direccion': 'direccion_proveedor', 'telefono': 'telefono_proveedor', 'email': 'email_proveedor'})
df_proveedor.head()

,idproveedor,tipo_persona,nombre_proveedor,tipo_documento_proveedor,num_documento_proveedor,direccion_proveedor,telefono_proveedor,email_proveedor
5,975294482,proveedor,Jessica Foster,DNI,197515122098189,"293 Brendan Alley Waltersville, MN 27396",583.498.,reneefletcher@gmail.com
8,128188438,proveedor,David Gould,Tarjeta Residencia,190081944389650,Unit 5345 Box 2954 DPO AA 26465,(403)231,jessica96@hotmail.com
9,296550422,proveedor,Timothy Smith,Pasaporte,124017249679379,"9903 Randy Wells South Sara, CT 22848",+1-424-6,smithemma@yahoo.com
10,479371286,proveedor,Norman Torres,Tarjeta Residencia,150258507110421,"02800 Thomas Ridge Eduardochester, MN 67312",(226)427,sanchezsandra@hotmail.com
12,513746963,proveedor,Richard Goodwin,Tarjeta Residencia,195187872647193,"7972 David Extension Apt. 215 Aguirreberg, UT ...",001-922-,jtownsend@gmail.com


In [35]:
df_ingreso_proveedor = df_ingreso.merge(df_proveedor, how='inner', on='idproveedor')
df_ingreso_proveedor.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado,tipo_persona,nombre_proveedor,tipo_documento_proveedor,num_documento_proveedor,direccion_proveedor,telefono_proveedor,email_proveedor
0,531619841,376765211,5692000,Ticket de Venta,1916930,None,2023-06-24,71.55,11050.94,Cancelada,proveedor,Darren Rogers,DNI,119072290728819,"652 Coleman Crossing Jacksonburgh, GA 53235",418-544-,derekparks@hotmail.com
1,815371920,376765211,1205369,Nota de Débito,9561773,None,2016-04-02,93.41,90210.75,Reembolsada,proveedor,Darren Rogers,DNI,119072290728819,"652 Coleman Crossing Jacksonburgh, GA 53235",418-544-,derekparks@hotmail.com
2,501905412,478520944,9255184,Ticket de Venta,1388548,None,2020-06-20,94.38,3490.30,Completada,proveedor,Elijah Kaufman,NIE,198102176013966,"PSC 7811, Box 3900 APO AE 87916",957.535.,mbrown@hotmail.com
3,460837727,478520944,1058503,Recibo,6116245,None,2016-02-01,25.17,19699.51,Pendiente,proveedor,Elijah Kaufman,NIE,198102176013966,"PSC 7811, Box 3900 APO AE 87916",957.535.,mbrown@hotmail.com
4,461116390,478520944,9923559,Factura,6923290,None,2023-04-07,92.82,34462.05,Pendiente,proveedor,Elijah Kaufman,NIE,198102176013966,"PSC 7811, Box 3900 APO AE 87916",957.535.,mbrown@hotmail.com


In [36]:
sql_query = 'SELECT * FROM usuario'
df_usuario = pd.read_sql(sql_query, postgres_driver)
df_proveedor = df_proveedor.rename(columns={'nombre': 'nombre_usuario', 'tipo_documento': 'tipo_documento_usuario', 'num_documento': 'num_documento_usuario', 'direccion': 'direccion_usuario', 'telefono': 'telefono_usuario', 'email': 'email_usuario'})
df_usuario.head()

,idusuario,idrol,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado
0,8818688,10003,Richard James,DNI,158643840073728,"222 Obrien Ferry West Williamland, MP 22317",515-922-,ikramer@yahoo.com,"[b'c', b'0', b'2', b'2', b'f', b'3', b'5', b'1...",1
1,7258115,10003,Amber Grant,Pasaporte,125079704406019,"76095 Wagner Ford Apt. 141 Ryanmouth, VI 68589",552-971-,smitheugene@gmail.com,"[b'd', b'8', b'7', b'5', b'd', b'c', b'7', b'8...",1
2,5206026,10002,Carolyn Jones,NIE,115369222688772,"85590 Pamela Lights Apt. 211 Joyceville, NJ 92480",+1-914-4,aalvarez@gmail.com,"[b'7', b'4', b'4', b'6', b'a', b'9', b'c', b'9...",0
3,2824202,10002,Brandy Bryant,NIE,133877029232646,"5870 Massey Grove Suite 074 Lake Jessicaville,...",+1-772-5,odonnellashley@yahoo.com,"[b'f', b'5', b'5', b'5', b'0', b'1', b'd', b'8...",0
4,4433933,10004,Tom Salinas,Pasaporte,137802450690055,"49057 Garcia Junctions Smithberg, KS 74217",(865)418,lisawhitaker@hotmail.com,"[b'1', b'c', b'3', b'9', b'9', b'3', b'4', b'6...",0


In [37]:
dimIngreso = df_ingreso_proveedor.merge(df_usuario, how='inner', on='idusuario')
dimIngreso.drop(['idproveedor', 'idusuario', 'idrol'], axis=1,  inplace=True)
dimIngreso.head()

,idingreso,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_x,tipo_persona,nombre_proveedor,...,telefono_proveedor,email_proveedor,nombre,tipo_documento,num_documento,direccion,telefono,email,clave,estado_y
0,531619841,Ticket de Venta,1916930,None,2023-06-24,71.55,11050.94,Cancelada,proveedor,Darren Rogers,...,418-544-,derekparks@hotmail.com,Hayden Christensen,NIE,181466685839993,"895 Patterson Mountains South Nicholas, MH 16706",001-377-,jonathan63@hotmail.com,"[b'e', b'7', b'd', b'9', b'0', b'8', b'7', b'0...",1
1,777517748,Nota de Débito,8469206,None,2022-06-04,98.91,16367.60,Cancelada,proveedor,Rebecca Bailey,...,001-220-,tdennis@yahoo.com,Hayden Christensen,NIE,181466685839993,"895 Patterson Mountains South Nicholas, MH 16706",001-377-,jonathan63@hotmail.com,"[b'e', b'7', b'd', b'9', b'0', b'8', b'7', b'0...",1
2,815371920,Nota de Débito,9561773,None,2016-04-02,93.41,90210.75,Reembolsada,proveedor,Darren Rogers,...,418-544-,derekparks@hotmail.com,Alex Rubio,NIE,141502647921807,"7444 Mariah Parkway Suite 599 East Brianberg, ...",(436)401,acunningham@yahoo.com,"[b'e', b'5', b'3', b'd', b'd', b'b', b'4', b'6...",1
3,501905412,Ticket de Venta,1388548,None,2020-06-20,94.38,3490.30,Completada,proveedor,Elijah Kaufman,...,957.535.,mbrown@hotmail.com,Mathew Fernandez,DNI,136949781372712,"41000 Bentley Ports Suite 570 Deleonmouth, FL ...",337-399-,ijones@yahoo.com,"[b'c', b'c', b'2', b'e', b'c', b'3', b'1', b'f...",0
4,460837727,Recibo,6116245,None,2016-02-01,25.17,19699.51,Pendiente,proveedor,Elijah Kaufman,...,957.535.,mbrown@hotmail.com,Susan Romero,NIE,149349047443152,"922 Jennings Crossing Grayland, VI 91245",553-531-,ualvarez@gmail.com,"[b'6', b'2', b'3', b'0', b'0', b'5', b'c', b'7...",1


#### Dimensión Detalle Venta

In [38]:
sql_query = 'SELECT * FROM detalle_venta;'
df_detalle_venta = pd.read_sql(sql_query, postgres_driver)
df_detalle_venta.head()

,iddetalle_venta,idventa,idarticulo,cantidad,precio,descuento
0,4915206,731618,543900,65,831.03,975.73
1,6651912,668353,242084,26,178.44,634.56
2,5242890,853455,947852,8,205.41,400.37
3,8945684,754750,478574,12,398.65,317.91
4,7274517,912490,833028,39,586.71,356.01


In [41]:
sql_query = 'SELECT * FROM venta;'
df_venta = pd.read_sql(sql_query, postgres_driver)
df_venta = df_venta.rename(columns={'estado': 'estado_venta'})
df_venta.head()

,idventa,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,804864,922565727,4919286,Nota de Crédito,7337987,13783041,2019-06-26,66.80,78626.93,Completada
1,223233,410170198,1642528,Nota de Crédito,6815748,13887489,2022-12-03,18.21,5128.87,Completada
2,727041,837562345,5222074,Recibo,9093126,68136964,2015-09-19,23.11,3700.99,Completada
3,647168,323201389,9489165,Recibo,9406470,47261701,2017-01-13,21.06,13558.78,Reembolsada
4,514056,350764427,8768679,Factura,8867849,61259781,2019-05-24,31.81,14911.36,Reembolsada


In [42]:
df_detalle_venta_venta = df_detalle_venta.merge(df_venta, how='inner', on='idventa')
df_detalle_venta_venta.head()

,iddetalle_venta,idventa,idarticulo,cantidad,precio,descuento,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta
0,4915206,731618,543900,65,831.03,975.73,613983304,9535608,Boleta de Venta,3344936,69835281,2016-05-01,58.0,24377.84,Cancelada
1,4463886,731618,864780,32,527.51,669.21,613983304,9535608,Boleta de Venta,3344936,69835281,2016-05-01,58.0,24377.84,Cancelada
2,4825318,731618,912347,25,979.95,754.80,613983304,9535608,Boleta de Venta,3344936,69835281,2016-05-01,58.0,24377.84,Cancelada
3,6405440,731618,989346,27,623.90,972.52,613983304,9535608,Boleta de Venta,3344936,69835281,2016-05-01,58.0,24377.84,Cancelada
4,6111089,731618,209683,81,403.27,708.77,613983304,9535608,Boleta de Venta,3344936,69835281,2016-05-01,58.0,24377.84,Cancelada


In [43]:
sql_query = 'SELECT *FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'estado': 'estado_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,299008,97521,4GlHzDRZ,Camisa Since,554.69,18,Artículo Camisa Since en venta en tienda,fig_0,1
1,417793,82016,BGkdWS9S,Pelota Hour,748.36,38,Artículo Pelota Hour en venta en tienda,fig_1,1
2,389124,13750,XFsyQfgJ,Pelota Just,514.84,52,Artículo Pelota Just en venta en tienda,fig_2,1
3,233476,51470,0VKtCJNX,Portátil Force,728.38,58,Artículo Portátil Force en venta en tienda,fig_3,1
4,827400,51470,GuqF2Y9Y,Juguete Style,444.59,92,Artículo Juguete Style en venta en tienda,fig_4,1


In [44]:
dimDetalleVenta = df_detalle_venta_venta.merge(df_articulo, how='inner', on='idarticulo')
dimDetalleVenta.drop(['idventa', 'idarticulo', 'idcliente', 'idusuario'], axis=1,  inplace=True)
dimDetalleVenta.head()

,iddetalle_venta,cantidad,precio,descuento,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_venta,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,4915206,65,831.03,975.73,Boleta de Venta,3344936,69835281,2016-05-01,58.00,24377.84,Cancelada,40879,yv7H3iV0,Silla Go,831.03,88,Artículo Silla Go en venta en tienda,fig_573,1
1,9934124,88,831.03,112.13,Factura,6600726,81084430,2022-07-10,29.98,77594.64,Cancelada,40879,yv7H3iV0,Silla Go,831.03,88,Artículo Silla Go en venta en tienda,fig_573,1
2,6259748,54,831.03,841.31,Recibo,9050906,74554126,2017-12-09,35.89,60934.99,Cancelada,40879,yv7H3iV0,Silla Go,831.03,88,Artículo Silla Go en venta en tienda,fig_573,1
3,6635256,37,831.03,739.01,Nota de Crédito,3606060,56079927,2020-04-25,32.01,74752.97,En proceso,40879,yv7H3iV0,Silla Go,831.03,88,Artículo Silla Go en venta en tienda,fig_573,1
4,3721607,75,831.03,626.36,Factura,9232546,40448130,2018-02-25,24.33,25388.05,Reembolsada,40879,yv7H3iV0,Silla Go,831.03,88,Artículo Silla Go en venta en tienda,fig_573,1


#### Dimensión Detalle Ingreso


In [45]:
sql_query = 'SELECT * FROM detalle_ingreso;'
df_detalle_ingreso = pd.read_sql(sql_query, postgres_driver)
df_detalle_ingreso.head()

,iddetalle_ingreso,idingreso,idarticulo,cantidad,precio
0,102447105,829433898,174135,91,62.49
1,708354049,319015775,580976,96,925.75
2,111448066,156932409,920018,29,208.63
3,426625030,552887074,238296,56,251.14
4,532056071,307867961,345409,9,430.07


In [46]:
sql_query = 'SELECT * FROM ingreso;'
df_ingreso = pd.read_sql(sql_query, postgres_driver)
df_ingreso = df_ingreso.rename(columns={'estado': 'estado_ingreso'})
df_ingreso.head()

,idingreso,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso
0,531619841,376765211,5692000,Ticket de Venta,1916930,None,2023-06-24,71.55,11050.94,Cancelada
1,281835522,742713186,2735407,Recibo,1511426,None,2019-10-17,66.65,74033.63,Reembolsada
2,501905412,478520944,9255184,Ticket de Venta,1388548,None,2020-06-20,94.38,3490.30,Completada
3,781879301,761635125,3604790,Nota de Débito,1478661,None,2017-06-05,33.96,89249.26,Completada
4,900507658,409456127,9170192,Boleta de Venta,2965510,None,2018-12-26,3.13,68756.65,Cancelada


In [47]:
df_detalle_ingreso_ingreso = df_detalle_ingreso.merge(df_ingreso, how='inner', on='idingreso')
df_detalle_ingreso_ingreso.head()

,iddetalle_ingreso,idingreso,idarticulo,cantidad,precio,idproveedor,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso
0,102447105,829433898,174135,91,62.49,619488373,7283585,Boleta de Venta,6027317,None,2019-01-14,82.27,78985.48,Completada
1,843272853,829433898,464131,46,170.93,619488373,7283585,Boleta de Venta,6027317,None,2019-01-14,82.27,78985.48,Completada
2,708354049,319015775,580976,96,925.75,981958646,5607576,Recibo,2675602,None,2019-05-17,1.86,50841.55,Reembolsada
3,111448066,156932409,920018,29,208.63,350870793,7138186,Boleta de Venta,5632352,None,2015-11-23,92.62,78789.40,Cancelada
4,554345437,156932409,878069,94,534.86,350870793,7138186,Boleta de Venta,5632352,None,2015-11-23,92.62,78789.40,Cancelada


In [48]:
sql_query = 'SELECT * FROM articulo;'
df_articulo = pd.read_sql(sql_query, postgres_driver)
df_articulo = df_articulo.rename(columns={'estado': 'estado_articulo'})
df_articulo.head()

,idarticulo,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,299008,97521,4GlHzDRZ,Camisa Since,554.69,18,Artículo Camisa Since en venta en tienda,fig_0,1
1,417793,82016,BGkdWS9S,Pelota Hour,748.36,38,Artículo Pelota Hour en venta en tienda,fig_1,1
2,389124,13750,XFsyQfgJ,Pelota Just,514.84,52,Artículo Pelota Just en venta en tienda,fig_2,1
3,233476,51470,0VKtCJNX,Portátil Force,728.38,58,Artículo Portátil Force en venta en tienda,fig_3,1
4,827400,51470,GuqF2Y9Y,Juguete Style,444.59,92,Artículo Juguete Style en venta en tienda,fig_4,1


In [49]:
dimDetalleIngreso = df_detalle_ingreso_ingreso.merge(df_articulo, how='inner', on='idarticulo')
dimDetalleIngreso.drop(['idingreso', 'idarticulo', 'idproveedor', 'idusuario'], axis=1,  inplace=True)
dimDetalleIngreso.head()

,iddetalle_ingreso,cantidad,precio,tipo_comprobante,serie_comprobante,num_comprobante,fecha,impuesto,total,estado_ingreso,idcategoria,codigo,nombre,precio_venta,stock,descripcion,imagen,estado_articulo
0,102447105,91,62.49,Boleta de Venta,6027317,None,2019-01-14,82.27,78985.48,Completada,20934,KrcdyMeO,Anillo Moment,62.49,27,Artículo Anillo Moment en venta en tienda,fig_32,1
1,929837852,98,62.49,Factura,1220378,None,2021-05-06,39.24,35335.78,Reembolsada,20934,KrcdyMeO,Anillo Moment,62.49,27,Artículo Anillo Moment en venta en tienda,fig_32,1
2,666603063,40,62.49,Recibo,8450204,None,2023-10-28,75.09,54977.28,Reembolsada,20934,KrcdyMeO,Anillo Moment,62.49,27,Artículo Anillo Moment en venta en tienda,fig_32,1
3,843272853,46,170.93,Boleta de Venta,6027317,None,2019-01-14,82.27,78985.48,Completada,32684,n2W2ZGOu,Tableta Think,170.93,56,Artículo Tableta Think en venta en tienda,fig_616,1
4,708354049,96,925.75,Recibo,2675602,None,2019-05-17,1.86,50841.55,Reembolsada,51470,XoBu3fLm,Maquillaje College,925.75,51,Artículo Maquillaje College en venta en tienda,fig_672,1


#### Dimensión Rol


In [50]:
sql_query = 'SELECT * FROM rol;'
dimRol = pd.read_sql(sql_query, postgres_driver)
dimRol.head()

,idrol,nombre,descripcion,estado
0,10001,Administrador,Este rol tiene acceso completo al sistema y pu...,1
1,10002,Moderador,Este rol tiene permisos para moderar contenido...,1
2,10003,Usuario Estándar,Este rol tiene acceso limitado y puede realiza...,1
3,10004,Soporte Técnico,Este rol proporciona soporte técnico y tiene a...,1
4,10005,Invitado,Este rol tiene acceso limitado y solo puede ve...,1


#### Dimensión Categoría


In [51]:
sql_query = 'SELECT * FROM categoria;'
dimCategoria = pd.read_sql(sql_query, postgres_driver)
dimCategoria.head()

,idcategoria,nombre,descripcion,estado
0,39582,Ropa y Accesorios,"Incluye prendas de vestir para hombres, mujere...",1
1,82016,Calzado,"Incluye zapatos formales, tenis, botas, sandal...",1
2,51470,Juguetes y Juegos,Una amplia gama de juguetes para ninos de toda...,1
3,75369,Electronicos,"Desde telefonos moviles, computadoras portatil...",1
4,20934,Mascotas,"Ofrece productos para el cuidado de mascotas, ...",1


#### Dimensión de Tiempo


In [52]:
sql_query = 'SELECT fecha FROM venta;'
dimFecha = pd.read_sql(sql_query, postgres_driver)
dimFecha.head()

,fecha
0,2019-06-26
1,2022-12-03
2,2015-09-19
3,2017-01-13
4,2019-05-24


In [53]:
dimFecha['year'] = pd.DatetimeIndex(dimFecha['fecha']).year
dimFecha['month'] = pd.DatetimeIndex(dimFecha['fecha']).month
dimFecha['quarter'] = pd.DatetimeIndex(dimFecha['fecha']).quarter
dimFecha['day'] = pd.DatetimeIndex(dimFecha['fecha']).day
dimFecha['week'] = pd.DatetimeIndex(dimFecha['fecha']).week
dimFecha['dayofweek'] = pd.DatetimeIndex(dimFecha['fecha']).dayofweek
#dimFechaHora['hour'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).hour
#dimFechaHora['minute'] = pd.DatetimeIndex(dimFechaHora['fecha_hora']).minute
dimFecha['is_weekend'] = dimFecha['dayofweek'].apply(lambda x: 1 if x > 5 else 0)
dimFecha.head()

/var/folders/t4/ltm_f1s92xd8rkv3v0nc5ll80000gn/T/ipykernel_22164/1606713047.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dimFecha['week'] = pd.DatetimeIndex(dimFecha['fecha']).week


,fecha,year,month,quarter,day,week,dayofweek,is_weekend
0,2019-06-26,2019,6,2,26,26,2,0
1,2022-12-03,2022,12,4,3,48,5,0
2,2015-09-19,2015,9,3,19,38,5,0
3,2017-01-13,2017,1,1,13,2,4,0
4,2019-05-24,2019,5,2,24,21,4,0


In [54]:
dimFecha['id_fecha'] = dimFecha['year'].astype(str) + dimFecha['month'].astype(str)
dimFecha['id_fecha'] = dimFecha['id_fecha'].astype(str) + dimFecha['day'].astype(str)
#dimFecha['id_fecha'] = dimFecha['id_fecha'].astype(str) + dimFechaHora['hour'].astype(str)
#dimFechaHora['id_fecha'] = dimFechaHora['id_fecha'].astype(str) + dimFechaHora['minute'].astype(str)
dimFecha.drop_duplicates(inplace=True)
dimFecha.head()

,fecha,year,month,quarter,day,week,dayofweek,is_weekend,id_fecha
0,2019-06-26,2019,6,2,26,26,2,0,2019626
1,2022-12-03,2022,12,4,3,48,5,0,2022123
2,2015-09-19,2015,9,3,19,38,5,0,2015919
3,2017-01-13,2017,1,1,13,2,4,0,2017113
4,2019-05-24,2019,5,2,24,21,4,0,2019524


#### Tabla de Hechos


In [55]:
sql_query = '''SELECT idventa, fecha, idcliente,
                              idusuario, tipo_comprobante, serie_comprobante, num_comprobante, impuesto, total,
                              estado FROM venta;'''
df_factTable = pd.read_sql(sql_query, postgres_driver)
df_factTable.head()

,idventa,fecha,idcliente,idusuario,tipo_comprobante,serie_comprobante,num_comprobante,impuesto,total,estado
0,804864,2019-06-26,922565727,4919286,Nota de Crédito,7337987,13783041,66.80,78626.93,Completada
1,223233,2022-12-03,410170198,1642528,Nota de Crédito,6815748,13887489,18.21,5128.87,Completada
2,727041,2015-09-19,837562345,5222074,Recibo,9093126,68136964,23.11,3700.99,Completada
3,647168,2017-01-13,323201389,9489165,Recibo,9406470,47261701,21.06,13558.78,Reembolsada
4,514056,2019-05-24,350764427,8768679,Factura,8867849,61259781,31.81,14911.36,Reembolsada
